In [1]:
import numpy as np
import pandas as pd
from clusterlogs import pipeline
import nltk

In [2]:
pd.set_option('display.max_colwidth', -1)

/home/nikodemas/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [3]:
df = pd.read_csv('error_dst_0105.txt')

In [4]:
df.head()

,time,error_message
0,1578189601783,DESTINATION dcgftp.usatlas.bnl.gov ERROR transfer error copy failed with mode push with error copy could not get the delegation id could not get proxy request error fault soap env server no subcode http not found detail
1,1578189607441,DESTINATION gfe02.grid.hep.ph.ic.ac.uk ERROR transfer transfer canceled because the gsiftp performance marker timeout of seconds has been exceeded or all performance markers during that period indicated zero bytes transferred
2,1578189609698,DESTINATION ccdavatlas.in2p3.fr ERROR transfer error copy failed with mode push with error copy http conflict file exist
3,1578189611931,DESTINATION charon01.westgrid.ca ERROR transfer error copy failed with mode push with error transfer failed failure problem sending data certificate for doesnt match any of the subject alternative names rptaylor@uvic.ca
4,1578189612498,DESTINATION fal-pygrid-30.lancs.ac.uk ERROR protocol not supported or path url invalid root pnfs usatlas.bnl.gov atlasscratchdisk rucio tests


In [5]:
cluster = pipeline.Chain(df, target='error_message', mode='create')

In [6]:
cluster.process()

Found 4400 equal groups
...Matching clusterization
Finished with 329 clusters


In [10]:
cluster.result

,pattern,tokenized_pattern,indices,cluster_size,sequence,common_phrases
11,DESTINATION ERROR srm ifce err communication error on send err se httpg srm cgsi gsoap running on reports error reading token data header connection,"[DESTINATION, ▁, ▁, ERROR, ▁, srm, ▁, ifce, ▁, err, ▁, communication, ▁, error, ▁, on, ▁, send, ▁, err, ▁, se, ▁, ▁, httpg, ▁, srm, ▁, cgsi, ▁, gsoap, ▁, running, ▁, on, ▁, reports, ▁, error, ▁, reading, ▁, token, ▁, data, ▁, header, ▁, connection, ▁]","[530501, 291415, 530338, 5015, 15442, 15740, 40870, 47501, 50061, 73453, 73643, 73687, 73694, 102263, 115060, 115308, 116662, 117199, 165924, 250394, 250550, 267673, 268506, 268622, 295609, 295703, 295814, 295889, 300350, 300399, 312405, 312647, 312833, 323896, 325922, 329679, 393651, 408010, 466809, 467013, 469713, 477548, 488414, 488690, 488818, 496467, 500656, 519273, 519709, 519753, 519755, 519814, 523458, 523640, 523689, 365134, 576355, 576367, 576371, 576382, 576396, 576402, 576419, 576427, 576441, 576445, 576456, 576457, 576470, 576474, 576485, 576496, 576505, 576506, 576519, 576522, 576540, 576541, 576543, 576547, 576554, 576584, 576589, 576591, 576595, 576597, 576598, 576599, 576609, 576611, 576613, 576620, 576621, 576622, 576629, 576634, 576635, 576636, 576637, 576648, ...]",55424,"[DESTINATION, ERROR, srm, ifce, err, communication, error, on, send, err, se, httpg, srm, cgsi, gsoap, running, on, reports, error, reading, token, data, header, connection]","[uk error srm ifce err communication error, ch error srm ifce err communication error, ca error srm ifce err communication error, es error srm ifce err communication error, pl error srm ifce err communication error, de error srm ifce err communication error, nl error srm ifce err communication error, kr error srm ifce err communication error, il error srm ifce err communication error, tw error srm ifce err communication error, cz error srm ifce err communication error, fr error srm ifce err communication error, ru error srm ifce err communication error, cn error srm ifce err communication error, org error srm ifce err communication error, su error srm ifce err communication error, ro error srm ifce err communication error, gr error srm ifce err communication error, sk error srm ifce err communication error, br error srm ifce err communication error, edu error srm ifce err communication error, gov error srm ifce err communication error, error srm ifce err communication error, destination lhcbsrm-disk-kit, destination dcache-se-cms, destination dcache-se-atlas, destination cms-lvs-gridftp, destination srm-cms, destination srm-lhcb, destination cms-t2-se01, destination srm-eoslhcb, destination storm-fe-lhcb, destination se-iep-grid, destination grid-srm, destination cms-gridftp, destination atlassrm-kit, destination srm, destination atlas-gridftp, destination tech-se, destination ruhex-osgce, destination f-dpm001, destination lapp-se01, destination lapp-testse01, destination srmlhcb-tape, destination storm-fe, destination storm-se-01, destination cmseos-gridftp, destination red-gridftp, destination lcg-se0, destination lcg-se1, destination grid-se, destination dc2-grid-64, edu, gov, destination se, destination se01, destination gridftp, destination storm, destination se1, destination srm01, destination lcgsedc01, destination eoscmsftp, destination srmcms, destination cmsio, destination dcsrm, destination srmatlas, destination atlasse, destination srmv2, destination cmsdcatape01, destination gfe02, destination gftp, destination golias100, destination ccsrm, destination lpnse1, destination hephyse, destination storage01, destination se03, destination cmsdcadisk01, destination sdrm, destination transfer, destination se0002, destination tbit00, destination grid02, destination lcgdpmse, destination charon01, destination t2cmcondor, destination hepgrid11, destination heplnx204, destination marsedpm, destination head01, destination eosatlassftp, destination lhcbsdrm, destination node12, uni-wuppertal, t2, l

Separating clusters containing fewer than 3 messages

In [7]:
clusters, outliers = cluster.split_clusters(cluster.result, 'cluster_size', threshold=3)


Top 10 clusters by the size of cluster

In [8]:
top_numb = 10
top_clusters = cluster.validation(clusters)[['cluster_size','pattern','mean_similarity']].sort_values(by=["cluster_size"], ascending=False) #clusters sorted by similarity inside the cluster
top_clusters.head(top_numb)

,cluster_size,pattern,mean_similarity
0,55424,DESTINATION ERROR srm ifce err communication error on send err se httpg srm cgsi gsoap running on reports error reading token data header connection,0.77
1,52867,DESTINATION ERROR error on xrdcl copyprocess run error server responded with an error write permission denied,0.89
2,42959,DESTINATION ERROR source srm get turl error on the turl request se srm file unavailable file is .,0.87
3,28596,DESTINATION ERROR globus ftp client the server responded with an error service busy connection limit exceeded. please try again later. closing control connection.,0.88
4,27279,DESTINATION ERROR error on xrdcl copyprocess run error server responded with an error authentication to failed all protocols have been tried.,0.92
5,24969,DESTINATION ERROR protocol not supported or path url invalid root atlasscratchdisk rucio tests,0.80
6,24441,DESTINATION ERROR error on xrdcl copyprocess run error server responded with an error unable to create permission denied,0.92
7,21696,DESTINATION ERROR transfer transfer canceled because the gsiftp performance marker timeout of seconds has been exceeded or all performance markers during that period indicated zero bytes transferred,0.91
8,20554,DESTINATION ERROR error reported from srm ifce se ls srm invalid path no such file or directory,0.89
9,18381,DESTINATION ERROR gfalt copy file perform copy error on xrdcl copyprocess run error server responded with an error,0.97


Most inaccurate clusters

In [9]:
top_clusters[['cluster_size','pattern','mean_similarity']].sort_values(by=["mean_similarity"]).head(5)

,cluster_size,pattern,mean_similarity
18,9370,DESTINATION ERROR operation timed out,0.65
61,1171,DESTINATION ERROR srm ifce err connection timed out err se etimedout httpg srm user timeout over,0.67
51,1605,DESTINATION ERROR srm operation canceled,0.68
57,1377,DESTINATION ERROR destination checksum operation timed out,0.70
84,666,DESTINATION ERROR globus ftp client the operation was aborted,0.71


In [16]:
print(cluster.in_cluster(cluster.result, 1)[[1,5,100]],"\n")

['DESTINATION eosatlashttp.cern.ch ERROR transfer error copy failed with mode push with error request cancellation was requested.'
 'DESTINATION webdav.grid.surfsara.nl ERROR transfer error copy failed with mode pull with error request cancellation was requested.'
 'DESTINATION xfer.cr.cnaf.infn.it ERROR transfer error copy failed with mode pull with error request cancellation was requested.'] 

